In [ ]:
TITLE = "mnist/swin"
print(f"Active {TITLE}")

In [ ]:
import argparse
import time
import numpy as np
import models
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torchvision import transforms
from timm.optim import create_optimizer
from timm.utils import accuracy, AverageMeter
from datasets.mnist.mnist_data import get_mnist_dataset
from util.show_images import ShowImages
from util.visualization import Visualization
from util.notification import Notification
from util.custom_time import Time
from util.path_functions import PathFunctions

path_functions_instance = PathFunctions()

In [ ]:
SAVEFILENAME = "mnist-swin"
SAVEWEIGHTPATH = f"dev/result/weight/{SAVEFILENAME}.pth"

In [ ]:
def get_args_parser():
  parser = argparse.ArgumentParser(description="Training Config")
  parser.add_argument('--batch-size', default=64, type=int)
  parser.add_argument('--epochs', default=300, type=int)
  parser.add_argument('--device', default='cuda:0',
                        help='device to use for training / testing')
  parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
  parser.add_argument('--resize', default=224, type=int)

  # Model parameters
  parser.add_argument("--input-size", default=None, nargs=3, type=int, help='images input size')

  # Optimizer parameters
  parser.add_argument('--opt', default='adamw', type=str, metavar='OPTIMIZER',
                      help='Optimizer (default: "adamw"')
  parser.add_argument('--opt-eps', default=1e-8, type=float, metavar='EPSILON',
                      help='Optimizer Epsilon (default: 1e-8)')
  parser.add_argument('--opt-betas', default=None, type=float, nargs='+', metavar='BETA',
                      help='Optimizer Betas (default: None, use opt default)')
  parser.add_argument('--clip-grad', type=float, default=None, metavar='NORM',
                      help='Clip gradient norm (default: None, no clipping)')
  parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                      help='SGD momentum (default: 0.9)')
  parser.add_argument('--weight-decay', type=float, default=0.05,
                      help='weight decay (default: 0.05)')

  # Learning rate schedule parameters
  parser.add_argument("--lr", type=float, default=0.01, metavar="LR")
  
  return parser


In [ ]:

args = get_args_parser().parse_args(["--input-size", "3", "224", "224"])
args.epochs = 10
args.resize = 224
args.lr = 0.001
Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = args.epochs
BATCH_SIZE = args.batch_size
RESEIZE = args.resize

In [ ]:
transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  # 1チャンネルを3チャンネルに変換
        transforms.Resize((RESEIZE, RESEIZE)),  # ViTモデルは大きな画像が必要
        transforms.ToTensor()
    ])
train_data_loader = get_mnist_dataset(train=True, transform=transform, batch_size=BATCH_SIZE)
test_data_loader = get_mnist_dataset(train=False, transform=transform, batch_size=BATCH_SIZE)
examples = next(iter(train_data_loader))


In [ ]:
show_images_instance = ShowImages()

show_images_instance.show_images_loader(dataset_lodaer=train_data_loader, num_images_to_display=30)

In [ ]:
# model = models.swin_base_4_224(pretrained=True,  num_classes=10)
model = models.swin_tiny_4_224(pretrained=True,  num_classes=10)

In [ ]:
model.to(args.device)
train_loss_fn = nn.CrossEntropyLoss().cuda()
validate_loss_fn = nn.CrossEntropyLoss().cuda()

optimizer = create_optimizer(args, model)

# Set up the learning rate scheduler (if needed)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
def convert_seconds(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return hours, minutes, seconds

def train_one_epoch(model, loader, optimizer, loss_fn, args):
  batch_time_m = AverageMeter()
  losses_m = AverageMeter()
  accuracy_m = AverageMeter()

  model.train()
  start = time.time()
  print("--Active train--")
  for _, (inputs, labels) in enumerate(loader):
      # 勾配をゼロにリセット
      optimizer.zero_grad()
      # 入力データとラベルをデバイスに移動
      inputs, labels = inputs.to(args.device), labels.to(args.device)
      # モデルに入力データを渡して予測を取得
      outputs = model(inputs)
      # 損失を計算
      loss = loss_fn(outputs, labels)
      # 勾配を計算
      loss.backward()
      # オプティマイザでパラメータを更新
      optimizer.step()

      # 精度の計算
      _, predicted = torch.max(outputs.data, 1)
      correct = (predicted == labels).sum().item()
      accuracy = correct / labels.size(0)
      accuracy_m.update(accuracy, labels.size(0))

      # 更新
      torch.cuda.synchronize()
      losses_m.update(loss.item(), inputs.size(0))
      batch_time_m.update(time.time() - start)
      start = time.time()
  train_accuracy = accuracy_m.avg * 100
  # バッチごとの時間を表示
  # バッチごとの時間を変換
  batch_time_sum_h, batch_time_sum_min, batch_time_sum_sec = convert_seconds(batch_time_m.sum)
  batch_time_avg_h, batch_time_avg_min, batch_time_avg_sec = convert_seconds(batch_time_m.avg)
  print(
      f"Batch Time: {batch_time_sum_h:.0f}h {batch_time_sum_min:.0f}min {batch_time_sum_sec:.4f}sec, "
      f"Batch Time Average: {batch_time_avg_h:.0f}h {batch_time_avg_min:.0f}min {batch_time_avg_sec:.4f}sec"
  )
  print(f"Accuracy: {train_accuracy:.2f}%, Loss: {losses_m.avg}")
  return {
      'accuracy': train_accuracy,
      'loss': losses_m.avg,
      'batch_time_sum': batch_time_m.sum,
      'batch_time_avg': batch_time_m.avg
  }

def test(model, loader, args):
    print('--Active test--')
    accuracy_m = AverageMeter()
    # モデルを評価モードに設定
    model.eval()
    # 勾配を計算しないようにするための torch.no_grad() コンテキストを使用
    with torch.no_grad():
      # データローダーからバッチごとのデータを取得
      for _, (inputs, labels) in enumerate(loader):
        # 入力データとラベルをデバイスに移動
        inputs, labels = inputs.to(args.device), labels.to(args.device)
        # モデルに入力データを渡して予測を取得
        output = model(inputs)
        # 精度の計算
        _, predicted = torch.max(output.data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)
        accuracy_m.update(accuracy, labels.size(0))
    # テスト精度を計算
    test_accuracy = accuracy_m.avg * 100
    print(f"Accuracy: {test_accuracy:.2f}%")
    return {'accuracy': test_accuracy}


In [ ]:
train_accuracy_history = []
train_loss_history = []
train_batch_time_history = []
train_batch_time_ave_history = []
test_accuracy_history = []

for epoch in range(args.start_epoch, args.epochs):
    print(f"Epoch [{epoch + 1}/{args.epochs}]")
    train_result = train_one_epoch(model, train_data_loader, optimizer, train_loss_fn, args)
    test_result = test(model, test_data_loader, args)

    # トレーニング結果を保存
    train_accuracy_history.append(train_result['accuracy'])
    train_loss_history.append(train_result['loss'])
    train_batch_time_history.append(train_result['batch_time_sum'])
    train_batch_time_ave_history.append(train_result['batch_time_avg'])

    # テスト結果を保存
    test_accuracy_history.append(test_result['accuracy'])

final_save_path = path_functions_instance.absolutePath(SAVEWEIGHTPATH) 
torch.save(model.state_dict(), final_save_path)

In [ ]:
# グラフのx軸にエポック数を+1して1エポック目から始まるように修正
x = list(range(1, args.epochs + 1))
x_ticks = list(range(1, args.epochs + 1, 10))
# グラフのプロット
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy_history, label="Train Accuracy")
plt.plot(test_accuracy_history, label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training and Test Accuracy")
plt.yticks(range(0, 101, 10))
plt.grid(True)
plt.show()

In [ ]:
# グラフのx軸にエポック数を+1して1エポック目から始まるように修正
x = list(range(1, args.epochs + 1))
x_ticks = list(range(1, args.epochs + 1, 10))
# グラフのプロット
plt.figure(figsize=(10, 5))
plt.plot(train_loss_history, label="Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training Loss")
plt.yticks(range(0, 10, 1))
plt.grid(True)
plt.show()

In [ ]:
Visualization_instance = Visualization()
chckpoint=SAVEWEIGHTPATH
Visualization_instance.position_embedding(checkpointPath=chckpoint, model=model)

In [ ]:
from util.visualization import Visualization
import models

Visualization_instance = Visualization()
model = model = models.vit_tiny_16_224(pretrained=False, num_classes=10)
checkpoint = SAVEWEIGHTPATH
image_path = "datasets/mnist/Image/sample_0.png"

Visualization_instance.attention_map(model=model, checkpointPath=checkpoint, imagePath=image_path)

In [ ]:
notificationInstants = Notification()
customTimeInstants = Time()
channel="vit-team"
message = (
    f"accuracy: {train_accuracy_history[-1]}%, loss: {train_loss_history[-1]}, accuracy: {test_accuracy_history[-1]}% "
    f"Alltime: {customTimeInstants.convert_time_text(np.sum(train_batch_time_history))}, "
    f"TimeAve: {customTimeInstants.convert_time_text(np.sum(train_batch_time_ave_history))}"
)
notificationInstants.send_slack_notification(message=f"SUCESS: Active {TITLE}", channel=channel)
notificationInstants.send_slack_notification(message=message, channel=channel)